# Evaluation code for competition

## Load libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

## Set variables

In [2]:
types = ['confirmed','deaths','recovered']

data_dir='Track1_Submitted/'

## Load in data files

### Submitter (predicted)

In [3]:
def get_submitter_dict(data_dir):
    unis = [x for x in os.listdir(data_dir) if '.DS' not in x]
    submitter_dict = {}
    for uni in unis:
        uni_dict = {}
        files = [x for x in os.listdir(data_dir+uni) if '.DS' not in x]
        file_dict = {}
        for type_ in types:
            f_list = [x for x in files if type_ in x]
            file_dict[type_] = f_list
            if len(f_list)>0:
                uni_dict[type_] = pd.read_csv(data_dir+uni+'/'+f_list[0])
        submitter_dict[uni] = uni_dict
    return submitter_dict

submitter_dict = get_submitter_dict(data_dir)

### JHU (true)

In [4]:
true_dte='5/1/20'
filt_dte='4/23/20'
prefix = 'data/time_series_covid19_'
suffix = '_global.csv'

def get_jhu_target(true_dte, filt_dte, fname):
    target_df = \
        (pd.
         read_csv(fname).
         rename(columns = {true_dte : 'true'}).
         rename(columns = {filt_dte : 'filt'}).
         loc[:,['Province/State','Country/Region','true', 'filt']].
         #only view countries/states with filt counts>100
         query('filt>100')
        )
    return target_df

def get_jhu_dict(true_dte, filt_dte, prefix, types, suffix):
    jhu_dict = {}
    for type_ in types:
        jhu_dict[type_] =  get_jhu_target(true_dte, filt_dte,  prefix+type_+suffix)
    return jhu_dict   

jhu_dict = get_jhu_dict(true_dte, filt_dte, prefix, types, suffix)

## Compare data files

In [5]:
pred_dte = '5/1/20'
results_dict = {}
compare_dfs = {}


            
def get_compare_df(df_s, df_j):            
    print(df_s.shape)
    print(df_j.shape)
    compare_df = \
        (df_j[['true']].
        join(df_s[[pred_dte]].
        rename(columns = {pred_dte : 'pred'}).
        assign(pred = lambda x: x.pred.where(x.pred.ge(0))),
        how='left')
        )
    print(compare_df.shape)
    return compare_df

def get_rmse(y_true, y_pred):
    return mean_squared_error(y_true,y_pred) ** 0.5

def get_log(col, compare_df):
    return (compare_df[col].
                      replace([-0,0],1).
                      replace([np.inf, -np.inf], np.nan).
                      replace([np.nan, -np.nan],1).
                      apply(np.log).
                      values)

def get_results_df(submitter_dict, jhu_dict ):
    for uni in submitter_dict.keys():
        if uni!='sl4646':
            print(uni)
            uni_dict = {}
            compare_dfs[uni] = {}
            for type_ in jhu_dict.keys():
                print(type_)
                compare_df = get_compare_df((submitter_dict[uni][type_].
                                             set_index(['Province/State','Country/Region'])),
                                            (jhu_dict[type_].
                                             set_index(['Province/State','Country/Region']))                )
                compare_dfs[uni][type_] = compare_df
                uni_dict[type_] = get_rmse(get_log('pred',compare_df), get_log('true',compare_df))
            results_dict[uni] = uni_dict
    return pd.DataFrame(results_dict).T

results_df = get_results_df(submitter_dict , jhu_dict )

lcb2165
confirmed
(264, 2)
(183, 2)
(183, 2)
deaths
(264, 2)
(50, 2)
(50, 2)
recovered
(250, 2)
(141, 2)
(141, 2)
manasi.sharma
confirmed
(264, 3)
(183, 2)
(183, 2)
deaths
(264, 3)
(50, 2)
(50, 2)
recovered
(250, 3)
(141, 2)
(141, 2)
dlm2202
confirmed
(264, 96)
(183, 2)
(183, 2)
deaths
(264, 96)
(50, 2)
(50, 2)
recovered
(250, 96)
(141, 2)
(141, 2)
mcg2208
confirmed
(264, 3)
(183, 2)
(183, 2)
deaths
(264, 3)
(50, 2)
(50, 2)
recovered
(250, 3)
(141, 2)
(141, 2)
a.saakyan
confirmed
(264, 1)
(183, 2)
(183, 2)
deaths
(264, 1)
(50, 2)
(50, 2)
recovered
(250, 1)
(141, 2)
(141, 2)
by2287
confirmed
(264, 3)
(183, 2)
(183, 2)
deaths
(264, 3)
(50, 2)
(50, 2)
recovered
(250, 3)
(141, 2)
(141, 2)


## Display RMSE ranking

In [6]:
display(results_df.sort_values('confirmed'))

,confirmed,deaths,recovered
dlm2202,0.133013,0.131129,0.243079
mcg2208,0.139514,0.162693,0.307704
manasi.sharma,0.240490,0.160922,3.263254
lcb2165,0.326818,0.218046,0.421490
a.saakyan,0.377213,0.245567,0.883555
by2287,0.702877,0.401629,1.509048


In [7]:
display(results_df.sort_values('deaths'))

,confirmed,deaths,recovered
dlm2202,0.133013,0.131129,0.243079
manasi.sharma,0.240490,0.160922,3.263254
mcg2208,0.139514,0.162693,0.307704
lcb2165,0.326818,0.218046,0.421490
a.saakyan,0.377213,0.245567,0.883555
by2287,0.702877,0.401629,1.509048


In [8]:
display(results_df.sort_values('recovered'))

,confirmed,deaths,recovered
dlm2202,0.133013,0.131129,0.243079
mcg2208,0.139514,0.162693,0.307704
lcb2165,0.326818,0.218046,0.421490
a.saakyan,0.377213,0.245567,0.883555
by2287,0.702877,0.401629,1.509048
manasi.sharma,0.240490,0.160922,3.263254


In [9]:
compare_dfs['dlm2202']['confirmed']

true     pred
Province/State               Country/Region                          
NaN                          Afghanistan                2335     1891
                             Albania                     782      824
                             Algeria                    4154     3830
                             Andorra                     745      781
                             Argentina                  4532     4535
                             Armenia                    2148     1944
Australian Capital Territory Australia                   106      108
New South Wales              Australia                  3030     3059
Queensland                   Australia                  1034     1059
South Australia              Australia                   438      452
Tasmania                     Australia                   221      240
Victoria                     Australia                  1371     1362
Western Australia            Australia                   551      559
NaN                          Austria                   15531    15862
                             Azerbaijan                 1854     1776
                             Bahrain                    3170     3007
                             Bangladesh                 8238     7635
                             Belarus                   14917    13205
                             Belgium                   49032    48655
                             Bolivia                    1229     1047
                             Bosnia and Herzegovina     1781     1608
                             Brazil                    92202    75646
                             Brunei                      138      156
                             Bulgaria                   1555     1431
                             Burkina Faso                649      710
                             Cambodia                    122      130
                             Cameroon                   1832     1827
Alberta                      Canada                     5573     5548
British Columbia             Canada                     2145     2241
Manitoba                     Canada                      281      286
...                                                      ...      ...
NaN                          Slovakia                   1403     1657
                             Slovenia                   1434     1489
                             Somalia                     601      658
                             South Africa               5951     5720
                             Spain                    213435   235613
                             Sri Lanka                   690      530
                             Sudan                       533      412
                             Sweden                    21520    21119
                             Switzerland               29705    29950
                             Taiwan*                     429      443
                             Tanzania                    480      470
                             Thailand                   2960     3138
                             Trinidad and Tobago         116      121
                             Tunisia                     998     1020
                             Turkey                   122392   130974
                             Ukraine                   10861    11024
                             United Arab Emirates      13038    12604
Channel Islands              United Kingdom              538      587
Gibraltar                    United Kingdom              144      138
Isle of Man                  United Kingdom              316      365
NaN                          United Kingdom           177454   168068
                             Uruguay                     648      632
                             US                      1103461  1028140
                             Uzbekistan                 2086     2205
                             Venezuela                   335      

In [10]:
compare_dfs['dlm2202']['deaths']

true   pred
Province/State Country/Region                  
NaN            Algeria               453    450
               Argentina             225    237
               Austria               589    648
               Bangladesh            170    223
               Belgium              7703   8144
               Brazil               6412   5422
Ontario        Canada               1265   1345
Quebec         Canada               2022   2090
NaN            Chile                 234    259
Hubei          China                4512   5049
NaN            Colombia              314    305
               Czechia               240    266
               Denmark               460    451
               Dominican Republic    313    342
               Ecuador              1063    789
               Egypt                 406    383
               Finland               218    305
               France              24563  24687
               Germany              6736   7780
               Greece                140    152
               Hungary               323    337
               India                1223   1197
               Indonesia             800    817
               Iran                 6091   6345
               Ireland              1265   1132
               Israel                225    261
               Italy               28236  28201
               Japan                 455    573
               Korea, South          250    259
               Mexico               1972   2065
               Morocco               171    220
               Netherlands          4893   5417
               Norway                210    263
               Pakistan              417    425
               Panama                192    205
               Peru                 1124    991
               Philippines           579    546
               Poland                651    642
               Portugal             1007   1214
               Romania               744    771
               Russia               1169   1159
               Saudi Arabia          169    229
               Serbia                179    160
               Spain               24543  26046
               Sweden               2653   2959
               Switzerland          1754   1777
               Turkey               3258   3508
               Ukraine               272    330
               United Kingdom      27510  23964
               US                  64943  66184

In [11]:
compare_dfs['dlm2202']['recovered']

true    pred
Province/State    Country/Region                        
NaN               Afghanistan                310     460
                  Albania                    488     537
                  Algeria                   1821    1979
                  Andorra                    468     489
                  Argentina                 1292    1425
                  Armenia                    977    1204
New South Wales   Australia                 2293    1730
Queensland        Australia                  965     980
South Australia   Australia                  422     410
Victoria          Australia                 1300    1334
Western Australia Australia                  511     422
NaN               Austria                  13110   13384
                  Azerbaijan                1365    1316
                  Bahrain                   1555    1609
                  Bangladesh                 174     473
                  Belarus                   2918    2350
                  Belgium                  11892   13466
                  Bosnia and Herzegovina     755     777
                  Brazil                   38039   45779
                  Brunei                     124     131
                  Bulgaria                   276     346
                  Burkina Faso               517     635
                  Cambodia                   120     123
                  Cameroon                   934    1393
                  Canada                   22764   22019
                  Chile                     9018    8314
Anhui             China                      985    1012
Beijing           China                      547     567
Chongqing         China                      573     606
Fujian            China                      353     382
...                                          ...     ...
NaN               Portugal                  1647    3027
                  Qatar                     1436    1737
                  Romania                   4328    4655
                  Russia                   13220   15683
                  Saudi Arabia              3555    4906
                  Senegal                    356     382
                  Serbia                    1343    1568
                  Singapore                 1268    2108
                  Slovakia                   558     538
                  Slovenia                   233     320
                  South Africa              2382    2472
                  Spain                   112050   98260
                  Sri Lanka                  162     171
                  Sweden                    1005     880
                  Switzerland              23900   24992
                  Taiwan*                    324     309
                  Thailand                  2719    3353
                  Tunisia                    316     416
                  Turkey                   53808   37872
                  Ukraine                   1413    1231
                  United Arab Emirates      2543    3532
Channel Islands   United Kingdom             406     382
Gibraltar         United Kingdom             131     149
Isle of Man       United Kingdom             271     286
NaN               Uruguay                    435     437
                  US                      164015  113615
                  Uzbekistan                1212    1149
                  Venezuela                  148     183
                  Vietnam                    219     259
                  Kosovo                     271     145

[141 rows x 2 columns]